本 notebook 是学习 Datewhale 的 llm-universe 所记录的操作过程以及部分笔记，课程地址是：https://github.com/datawhalechina/llm-universe/tree/main

#### 如何评估 LLM 应用
1. ***开发流程和验证迭代的核心：***
    * 大模型应用程序开发强调验证迭代。通过快速定义和调整 Prompt，开发者可以在很短的时间内得到反馈并做出改进。
    * 初期开发中，使用少量样本调整 Prompt 是常见的做法。
    * 随着测试的进行，会遇到一些无法通过当前 Prompt 解决的棘手例子，这些例子需要逐步加入到测试集中进行优化。
2. ***评估和优化方法：***
    * 通过不断添加 Bad Case（表现不佳的例子）到测试集来逐步扩大验证集，并开发性能指标进行自动评估。
    * 当系统性能达到预期时，可以随时停止优化过程。
3. ***自动化评估：***
    * 对于简单任务，评估较为容易实现。但复杂生成任务的评估需要更加细致的方法。
    * 通过不断优化 Prompt 或检索性能，可以逐步提升系统的整体表现。
4. ***RAG（Retrieval-Augmented Generation）范式评估：***
    * 基于 RAG 范式的应用程序包括两个核心部分：检索和生成。
    * 评估和优化也需要分别聚焦这两个部分，提升系统的检索精度和生成质量。

5. ***Bad Case 处理：***
    * 识别 Bad Case 并进行针对性优化是提升系统性能的重要步骤。
    * 优化过程中需要确保不会在原先表现良好的样例上出现失误。

#### 大模型评估方法
 1. 人工评估：在系统开发的初期，验证集体量较小，最简单、直观的方法即为人工对验证集中的每一个验证案例进行评估。    
 2. 简单自动评估
 3. 使用大模型评估
 4. 混合评估

In [1]:
### 人工评估的一般方法， 使用的模型是 glm-4
import sys
sys.path.append('/workspaces/llm-uni/llm-universe/notebook/C3 搭建知识库')

from zhipuai_embedding import ZhipuAIEmbeddings
from zhipuai_llm import ZhipuAILLM
from langchain.vectorstores.chroma import Chroma
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv, find_dotenv
import os

_ = load_dotenv(find_dotenv())    # read local .env file
zhipuai_api_key = os.environ['ZHIPUAI_API_KEY']

# 定义 Embeddings
embedding = ZhipuAIEmbeddings()
# 向量数据库持久化路径
persist_directory = '/workspaces/llm-uni/note/data_base'
# 加载数据库
vectordb = Chroma(
    persist_directory=persist_directory,  # 允许我们将persist_directory目录保存到磁盘上
    embedding_function=embedding
)
llm = ZhipuAILLM()

#### 工具使用介绍：
1. ```PromptTemplate ```是 LangChain 提供的一个模块，用于定义和管理提示（Prompt）。提示是你与语言模型交互的核心，通过提示，你可以定义你想要从模型中获得什么样的输出。
    * 一般用法： ```prompt = PromptTemplate(template=template, input_variables=["text"])```
    * 参数说明： 1) template： 模板字符串。     
                2）input_variables: 模板中使用的变量名列表。定义哪些变量将被传递给 format 方法。

2. ```RetrievalQA``` 是一个更高级的模块，用于构建基于检索的问答系统。它结合了信息检索和生成式问答模型的功能，能够从大规模的文本数据中检索相关信息并生成回答。
    * 一般用法： ```qa_chain = RetrievalQA(retriever=None, llm=llm, prompt_template=None)```
    * 参数说明： 1） retriever: 用于检索相关文档的对象。       
                2) llm: 用于生成回答的语言模型。
                3) prompt_template:  用于生成回答的提示模板。    
                4) return_source_documents: 布尔值，指示是否返回源文档。


In [2]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA


template_v1 = """使用以下上下文来回答最后的问题。如果你不知道答案，就说你不知道，不要试图编造答
案。最多使用三句话。尽量使答案简明扼要。总是在回答的最后说“谢谢你的提问！”。
{context}
问题: {question}
"""

QA_CHAIN_PROMPT = PromptTemplate(input_variables=["context","question"],
                                 template=template_v1)



qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=vectordb.as_retriever(),
                                       return_source_documents=True,
                                       chain_type_kwargs={"prompt":QA_CHAIN_PROMPT})

print("问题一：")
question = "南瓜书和西瓜书有什么关系？"
result = qa_chain({"query": question})
print(result["result"])

print("问题二：")
question = "应该如何使用南瓜书？"
result = qa_chain({"query": question})
print(result["result"])

问题一：


/opt/conda/envs/llm-universe/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


南瓜书是对周志华教授的《机器学习》（西瓜书）中部分公式和概念进行解析和补充的辅助教材。它旨在为那些想深入了解公式推导细节的读者提供帮助，以西瓜书的内容为基础，对其中较难理解的公式进行讲解和推导。南瓜书建议与西瓜书配合使用，作为学习过程中的参考。

谢谢你的提问！
问题二：
应该以西瓜书为主线，遇到推导不出或看不懂的公式时查阅南瓜书；对于初学者，不建议深究西瓜书第1章和第2章的公式，学到一定程度再回头研究；南瓜书内容以本科数学基础视角讲解，复杂数学知识可通过附录和参考文献深入学习。谢谢你的提问！


其中人工评估要遵从一些准则：
 1. 量化评估    
 2. 多维评估
简单自动化评估的方法：
 1. 构造客观题
 2. 计算相似度答案

In [3]:
prompt = '''
你是一个模型回答评估员。
接下来，我将给你一个问题、对应的知识片段以及模型根据知识片段对问题的回答。
请你依次评估以下维度模型回答的表现，分别给出打分：

① 知识查找正确性。评估系统给定的知识片段是否能够对问题做出回答。如果知识片段不能做出回答，打分为0；如果知识片段可以做出回答，打分为1。

② 回答一致性。评估系统的回答是否针对用户问题展开，是否有偏题、错误理解题意的情况，打分分值在0~1之间，0为完全偏题，1为完全切题。

③ 回答幻觉比例。该维度需要综合系统回答与查找到的知识片段，评估系统的回答是否出现幻觉，打分分值在0~1之间,0为全部是模型幻觉，1为没有任何幻觉。

④ 回答正确性。该维度评估系统回答是否正确，是否充分解答了用户问题，打分分值在0~1之间，0为完全不正确，1为完全正确。

⑤ 逻辑性。该维度评估系统回答是否逻辑连贯，是否出现前后冲突、逻辑混乱的情况。打分分值在0~1之间，0为逻辑完全混乱，1为完全没有逻辑问题。

⑥ 通顺性。该维度评估系统回答是否通顺、合乎语法。打分分值在0~1之间，0为语句完全不通顺，1为语句完全通顺没有任何语法问题。

⑦ 智能性。该维度评估系统回答是否拟人化、智能化，是否能充分让用户混淆人工回答与智能回答。打分分值在0~1之间，0为非常明显的模型回答，1为与人工回答高度一致。

你应该是比较严苛的评估员，很少给出满分的高评估。
用户问题：
```
{}
```
待评估的回答：
```
{}
```
给定的知识片段：
```
{}
```
你应该返回给我一个可直接解析的 Python 字典，字典的键是如上维度，值是每一个维度对应的评估打分。
不要输出任何其他内容。
'''

In [5]:
### 
from zhipuai import ZhipuAI

client = ZhipuAI(
    api_key=os.environ["ZHIPUAI_API_KEY"]
)

def gen_glm_messages(prompt):
    '''
    构造 glm 模型请求参数 messages
    
    请求参数：
        prompt: 对应的用户提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="glm-4", temperature = 0.1):
    '''
    获取 glm 模型调用结果

    请求参数：
        prompt: 对应的提示词
        model: 调用的模型
        temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~2。温度系数越低，输出内容越一致。
    '''
    messages = gen_glm_messages(prompt)
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

question = "应该如何使用南瓜书？"
result = qa_chain({"query": question})
answer = result["result"]
knowledge = result["source_documents"]
response = get_completion(prompt.format(question, answer, knowledge))
response

'```python\n{\n    "知识查找正确性": 1,\n    "回答一致性": 1,\n    "回答幻觉比例": 1,\n    "回答正确性": 0.9,  # 假设有一定程度的正确性，但可能没有完全涵盖所有细节或可能存在微小错误\n    "逻辑性": 0.9,    # 假设逻辑基本连贯，但可能存在一些不够严谨的地方\n    "通顺性": 0.8,    # 假设回答大体通顺，但可能存在一些语法或表达上的小问题\n    "智能性": 0.7     # 假设回答具有一定智能性，但可能仍有一些机器回答的特征\n}\n```'

In [6]:
### 使用 GLM-4-0520 模型
def gen_glm_messages(prompt):
    '''
    构造 glm 模型请求参数 messages
    
    请求参数：
        prompt: 对应的用户提示词
    '''
    messages = [{"role": "user", "content": prompt}]
    return messages


def get_completion(prompt, model="GLM-4-0520", temperature = 0.1):
    '''
    获取 glm 模型调用结果

    请求参数：
        prompt: 对应的提示词
        model: 调用的模型
        temperature: 模型输出的温度系数，控制输出的随机程度，取值范围是 0~2。温度系数越低，输出内容越一致。
    '''
    messages = gen_glm_messages(prompt)
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    if len(response.choices) > 0:
        return response.choices[0].message.content
    return "generate answer error"

question = "应该如何使用南瓜书？"
result = qa_chain({"query": question})
answer = result["result"]
knowledge = result["source_documents"]
response = get_completion(prompt.format(question, answer, knowledge))
response

'```python\n{\n    "知识查找正确性": 1,\n    "回答一致性": 1,\n    "回答幻觉比例": 1,\n    "回答正确性": 1,\n    "逻辑性": 1,\n    "通顺性": 1,\n    "智能性": 0.9\n}\n```'

内容还在完善中，会马上补充相关内容。